In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
import json
from gezi import tqdm
import gezi
import qgrid
import pymp
from multiprocessing import Pool, Manager, cpu_count 
from collections import Counter

In [2]:
def gtime(timestamp):
  x = time.localtime(timestamp)
  return f'{x.tm_mon}-{x.tm_mday}'

In [3]:
def deal(mark):
  dc = gezi.read_parquet(f'../input/{mark}/context.parquet')
  dids = set(dc.did.values)
  vids = set(dc.vid.values)
  dc['id'] = dc.did + '\t' + dc.vid.astype(str)  
  ids = set(dc.id)
  du = gezi.read_parquet(f'../input/{mark}/user.parquet')
  du['watches'] =  du.watch.apply(json.loads)
  now_time = gtime(dc.timestamp.values[0])
  did_vids = []
  for _, row in tqdm(du.iterrows(), total=len(du)):
    did = row['did']
    watches = row['watches']
    for timestamp, vid in watches:
      if gtime(timestamp) == now_time:
        id_ = did + str(vid)
        if vid in vids and id_ not in ids:
          did_vids += [(did, vid, timestamp)]
  dids, vids, timestamps = zip(*did_vids)
  dvs = pd.DataFrame({'did': dids, 'vid': vids, 'timestamp': timestamps})
  dc_vid = dc[['vid', 'prev']].drop_duplicates()
  m = {}
  for _, row in dc_vid.iterrows():
    vid, prev = row['vid'], row['prev']
    if vid not in m:
      m[vid] = Counter()
    m[vid][prev] += 1
  prevs = []
  for vid in dc_vid.vid:
    prevs += [m[vid].most_common(1)[0][0]]
  dc_vid = pd.DataFrame({'vid': dc_vid.vid.values, 'prev': prevs})
  dc_vid = dc_vid.drop_duplicates()
  dvs = dvs.merge(dc_vid, on='vid', how='left')
  dc_did = dc.drop(['label', 'index', 'prev', 'id', 'timestamp', 'vid'], axis=1)
  dc_did = dc_did.drop_duplicates()
  dc_did = dc_did.groupby('did', as_index=False).first()
  dvs = dvs.merge(dc_did, on='did', how='left')
  dvs['label'] = 1
  dvs['index'] = -1
  dc = dc.drop(['id'], 1)
  dvall = pd.concat([dc, dvs])
  dvall.to_csv(f'../input/{mark}/context.csv', index=False)

In [4]:
marks = [f'train/part_{i + 1}' for i in range(30)] 
ps = min(len(marks), cpu_count())
with pymp.Parallel(ps) as p:
  for i in tqdm(p.range(len(marks)),desc='convert'):
    deal(marks[i])
    print(f'done {i + 1}')

100%|██████████| 92104/92104 [01:01<00:00, 1508.90it/s]


done 19


convert: 100%|██████████| 1/1 [03:14<00:00, 194.97s/it]


done 18


convert: 100%|██████████| 1/1 [03:15<00:00, 195.17s/it]


...

convert: 100%|██████████| 1/1 [04:01<00:00, 241.52s/it]


done 26


convert: 100%|██████████| 1/1 [04:08<00:00, 248.23s/it]


done 25


convert: 100%|██████████| 1/1 [04:08<00:00, 248.70s/it]


In [8]:
dce = gezi.read_parquet(f'../input/eval/context.parquet')
dce.to_csv('../input/eval/context.csv', index=False)

In [7]:
len(dce)

5565249